# 1 Introduction
First, we have to activate the environment. The first run may take a while as external libraries will be downloaded and compiled.

In [1]:
Pkg.activate(".")

  Activating project at `~/code/OpenScience21.jl`


# 2. The Algorithm
In this notebook, I will describe the training process on a real device and a noisy simulator. Technically the algorithm is the same as in the notebook [04_algorithm_evaluation](04_algorithm_evaluation.ipynb) except for the use of the error mitigation technic which was not necessary on the simulator written in Julia. 

We will use here the codes described in the notebooks:
* [01_solution](01_solution.ipynb)
* [03_trotter_step](03_trotter_step.ipynb)
* [04_algorithm_evaluation](04_algorithm_evaluation.ipynb)
Therefore in the beginning we import the script with includes all necessary functions.

In [2]:
# The init file contains useful scripts and imports as well as connections to the IBM Qiskit account.
include("src/Challenge/init.jl")

evaluate (generic function with 1 method)

First, we have to find the parameter for the U4 gate in trotter steps (see [03_trotter_step](03_trotter_step.ipynb)), at the same time we generate the whole circuit with ten trotter steps, this is our goal to achieve.

In [3]:
t = π
trotter_steps = 10
qc_full, params, params2, params3 = generate_circuit(trotter_steps, trotter_steps, t, init=true)
addMeasuresOS(qc_full)
# Execute simulation
check_simulation_err(qc_full, t)
qc_full

                                                                             »
 q_0: ───────────────────────────────────────────────────────────────────────»
      ┌───────────────────────┐                             ┌───┐┌──────────┐»
 q_1: ┤ U3(π,-4.3798,-1.7351) ├─────────────────────────────┤ X ├┤ Rz(8π/5) ├»
      └───────────────────────┘                             └─┬─┘└──────────┘»
 q_2: ────────────────────────────────────────────────────────┼──────────────»
                ┌───┐             ┌──────────────────────┐    │  ┌──────────┐»
 q_3: ──────────┤ X ├─────────────┤ U3(π,0.45277,4.6676) ├────■──┤ Ry(7π/5) ├»
                └───┘             └──────────────────────┘       └──────────┘»
 q_4: ───────────────────────────────────────────────────────────────────────»
                ┌───┐          ┌───────────────────────────┐                 »
 q_5: ──────────┤ X ├──────────┤ U3(6.2835,0.084986,0.398) ├─────────────────»
                └───┘          └────────────────────

The below method compares the results of the execution of two circuits.

In [4]:
function check_circuits_err(qc1, qc2)
    sym1 = execute(backend, qc1)
    sym2 = execute(backend, qc2)

    sum(abs.(sym1 - sym2))
end

check_circuits_err (generic function with 1 method)

As was described in [01_solution](01_solution.ipynb) we have to iterate 5 times the QML algorithm to find the best ansact parameter. So let's begin :)

# 3. Step 1

In [5]:
const st = 2
step1_check_qc, _, _, _ = generate_circuit(trotter_steps, 8, t, params, params2, params3, init=false)
step1_step_qc_exp, _, _, _ = generate_circuit(trotter_steps, st, t, params, params2, params3, init=true)
step1_step_qc_exp

                                                                            »
q_0: ───────────────────────────────────────────────────────────────────────»
     ┌───────────────────────┐                             ┌───┐┌──────────┐»
q_1: ┤ U3(π,-4.3798,-1.7351) ├─────────────────────────────┤ X ├┤ Rz(8π/5) ├»
     └───────────────────────┘                             └─┬─┘└──────────┘»
q_2: ────────────────────────────────────────────────────────┼──────────────»
               ┌───┐             ┌──────────────────────┐    │  ┌──────────┐»
q_3: ──────────┤ X ├─────────────┤ U3(π,0.45277,4.6676) ├────■──┤ Ry(7π/5) ├»
               └───┘             └──────────────────────┘       └──────────┘»
q_4: ───────────────────────────────────────────────────────────────────────»
               ┌───┐          ┌───────────────────────────┐                 »
q_5: ──────────┤ X ├──────────┤ U3(6.2835,0.084986,0.398) ├─────────────────»
               └───┘          └───────────────────────────┘     

We have to define our step ansact.

In [6]:
step1_ansact = generate_ansact()

                                                                  »
q_0: ─────────────────────────────────────────────────────────────»
     ┌──────────────────────────┐                                 »
q_1: ┤ U3(2.0202,1.1016,2.3091) ├──────────────────■──────────────»
     └──────────────────────────┘                  │              »
q_2: ──────────────────────────────────────────────┼──────────────»
     ┌──────────────────────────┐                ┌─┴─┐            »
q_3: ┤ U3(2.0202,1.1016,2.3091) ├──■─────────────┤ X ├────────────»
     └──────────────────────────┘  │             └───┘            »
q_4: ──────────────────────────────┼──────────────────────────────»
     ┌──────────────────────────┐┌─┴─┐┌──────────────────────────┐»
q_5: ┤ U3(2.0202,1.1016,2.3091) ├┤ X ├┤ U3(2.0145,1.4682,3.5877) ├»
     └──────────────────────────┘└───┘└──────────────────────────┘»
q_6: ─────────────────────────────────────────────────────────────»
                                                

In the definition of the start parameter, we can do this twofold, first, we can rand it, the second, we can use the previously found parameter (for instance if we break the optimization process).

In [7]:
#step1_start_params = [6.490463894513841, 3.434943430067894, 5.375354561541277, 4.7906997798464115, 2.849325024133124, 2.7673620360366824, 1.4940499436354768, 4.487752591524359, 3.2948988322626382, 2.0225225509500486, -1.2343170007923738, 1.0431091490261049, 1.8837296276360387, 1.2169677999557502, 5.940807695305212, 5.421210358780618, 0.3173688878579456, -6.965567540712155, 4.835967484868803, 3.0127993733543517, 4.4620544822135475, 1.378337396849606, 1.980959799643362, 3.6789144092845376, 5.417860886313657, 1.8687391611628605, 5.092938820960128]
step1_start_params = getRandParameters(step1_ansact)
nothing

Let's generate circuit to optimization.

In [8]:
# Set parameters
setparameters!(step1_ansact, step1_start_params)

# Generate step qc
step1_qc = generate_empty_circuit()
# append
append!(step1_qc, step1_step_qc_exp)
append!(step1_qc, step1_ansact)
# Add measures
addMeasuresOS(step1_qc)
step1_qc

                                                                             »
 q_0: ───────────────────────────────────────────────────────────────────────»
      ┌───────────────────────┐                             ┌───┐┌──────────┐»
 q_1: ┤ U3(π,-4.3798,-1.7351) ├─────────────────────────────┤ X ├┤ Rz(8π/5) ├»
      └───────────────────────┘                             └─┬─┘└──────────┘»
 q_2: ────────────────────────────────────────────────────────┼──────────────»
                ┌───┐             ┌──────────────────────┐    │  ┌──────────┐»
 q_3: ──────────┤ X ├─────────────┤ U3(π,0.45277,4.6676) ├────■──┤ Ry(7π/5) ├»
                └───┘             └──────────────────────┘       └──────────┘»
 q_4: ───────────────────────────────────────────────────────────────────────»
                ┌───┐          ┌───────────────────────────┐                 »
 q_5: ──────────┤ X ├──────────┤ U3(6.2835,0.084986,0.398) ├─────────────────»
                └───┘          └────────────────────

Now, we have to decide which device we will use in the notebook: Julia Simulator, Noisy simulator, or a real device.

In [9]:
@enum ExecutePlace NoisySim RealDevice JuliaSim
#useExecutePlace = NoisySim
useExecutePlace = JuliaSim

JuliaSim::ExecutePlace = 2

Let's define some helper functions. One of the arguments passed to gradient descent method is Optimization Function, we have two types of these functions. One is dedicated to Julia simulator where we can use Zygote to calculate derivative, the secound one use parametr shift rules (and we run the whole circuit multiple times to mitigate the stochastic error).

In [10]:
# 
function create_julia_op(step_qc)
    step_loss(params) = loss_expected_zero_state(execute(backend, step_qc, params))
    step_dloss(params) = real(step_loss'(params))
    step_of = OptimizationFunction(false, (x) -> (step_loss(x), step_dloss(x)), step_loss)
    
    return step_of
end

function call_N_qderivative(step_qc, use_backend, x, N)
    y  = 0.0
    dy = zeros(27)
    for i in 1:N
        tmp_y, tmp_dy = qderivative(use_backend, step_qc, loss_expected_zero_state, x)
        y = y + tmp_y
        dy = dy + tmp_dy
    end
    return y/N, dy/N
end

function call_N_loss_expected_zero_state(step_qc, use_backend, x, N)
    y  = 0.0
    for i in 1:N
        tmp_y = qexecute(use_backend, step_qc, loss_expected_zero_state, x)
        y = y + tmp_y
    end
    return y/N
end


function create_dev_op(step_qc, use_backend)
#     step_of = OptimizationFunction(
#                     false,
#                     (x) -> qderivative(use_backend, step_qc, loss_expected_zero_state, x),
#                     (x) -> loss_expected_zero_state(execute(use_backend, setAndConvert(step_qc, x))))
    step_of = OptimizationFunction(
            false,
            (x) -> call_N_qderivative(step_qc, use_backend, x, 30),
            (x) -> call_N_loss_expected_zero_state(step_qc, use_backend, x, 30))

    return step_of
end

function create_op(use_execute_place, step_qc)
    if use_execute_place == JuliaSim   
        step_of = create_julia_op(step_qc)
    else
        if use_execute_place == NoisySim
            useBackend = qiskitBackendNoisySim
        else 
            useBackend = qiskitBackendJakarta
        end

        step_of = create_dev_op(step_qc, useBackend)
    end
    
    return step_of
end

create_op (generic function with 1 method)

In [11]:
step1_of = create_op(useExecutePlace, step1_qc)
nothing

In [12]:
val, xparams, itr = gradientDescent(step1_of, step1_start_params, α=0.01, maxItr=200,
                              argsArePeriodic=true, isExpectedZero=true, ϵ=1e-6, debug=true, useBigValInc=true)

4.736077930550223, 1.4756463792701489, 1.7957066462696345, 1.5063415652340704, 4.585675845553053, 2.4048968044745176, 4.1265215629538385, 3.8373135769259386, 1.6161413819329837, 4.05820363843742, 3.644372678990201, 6.2313102830314335, 0.8401297581036028, 2.109259472806518, 2.8080858545519116, 1.8571920171406087, 4.054817211769463, 1.4826316694445092, 1.2623835359392706, 5.759704593098012, 1.6978455366800518, 0.13909623609722296, 1.679437397152096, 3.8889288636049213, 5.070056890889753, 2.205484490483977, 2.6152333524804336
Start iteration 1, value: 1.8085070233923601, best: 1.8085070233923601, dval: 1.6739816047816864
4.734352471715189, 1.458906563222332, 1.7915619828265446, 1.4954153694154046, 4.591905897543369, 2.4046799187817007, 4.122356318372853, 3.8358782525596373, 1.6205033142568703, 4.057392158944511, 3.651825178658277, 6.214570466983616, 0.8517785920610428, 2.11811434512662, 2.8081110573003016, 1.8549994943150354, 4.054817211769463, 1.4900841691125855, 1.263006382027331, 5.759

4.527152948390175, 0.6709678370024569, 1.4232594446999887, 1.5246572731578645, 4.833517405295842, 2.3726303834642954, 4.445763385169964, 2.9525992726138055, 1.9419402409588524, 4.199143582951572, 3.7837940119514437, 5.413305670259471, 1.5911036684644537, 2.345898417482628, 2.8090733378044543, 1.7801170020405084, 4.054817211769463, 1.622053002405777, 1.516482282108783, 5.730940056271796, 2.1556447570604935, 0.5205835585147051, 1.679437397152096, 3.8263135072171717, 5.343082353589327, 2.205484490483977, 2.597600652699384
Start iteration 18, value: 0.08509208572128944, best: 0.08509208572128944, dval: 0.06843889556876631
4.61709878259684, 0.6763935886047553, 1.4229653883372635, 1.523730747049413, 4.834491119020631, 2.37423318741434, 4.446319692382644, 2.91852825126756, 1.9556409851785332, 4.209907937622595, 3.766648274165023, 5.429828867555704, 1.5739376366227162, 2.3533320302361362, 2.809519816763537, 1.7808575827705675, 4.054817211769463, 1.6049072646193612, 1.5166451896946835, 5.720981

4.7611805028223255, 0.6726260712110672, 1.2412611418750599, 1.3634772169301819, 4.8162366158706655, 2.376037828930395, 4.663376708080278, 2.772650535348519, 1.9388114537967476, 4.4068757509045495, 3.5830173837491976, 5.415422529865098, 1.4606858224920312, 2.7663021576032554, 2.797731772433157, 1.8095309539048579, 4.054817211769463, 1.4212763742034813, 1.8024978741172435, 5.592008181497455, 2.0707407542787193, 0.7966256181894275, 1.679437397152096, 3.7985094192636177, 5.493728941047312, 2.205484490483977, 2.449682971802921
Start iteration 40, value: 0.012112661749424675, best: 0.012112661749424675, dval: 0.038639874974033483
Start iteration 41, value: 0.01390616496375058, best: 0.012112661749424675, dval: 0.07668206665275479
Big value increase: val: 0.01390616496375058, e.val: 0.012112661749424675, max(α): 5.720182406960866
4.76118073645972, 0.6726254857173455, 1.2405500086415382, 1.360183016129874, 4.816359645009273, 2.3760420321471662, 4.663328784594116, 2.7728859106047175, 1.93880816

4.779253011795913, 0.6726134919157212, 1.09112214053348, 1.3416922113121745, 4.816405346653649, 2.3760561106864233, 4.788711644666788, 2.8160307117828722, 1.9388266469509972, 4.401305206844748, 3.5478977848988036, 5.415410169559471, 1.4720695934731038, 2.821907750263969, 2.7977316834095602, 1.6691928714369137, 4.054817211769463, 1.3861567753529902, 1.9618815833686514, 5.3818613293117386, 1.9925488560668088, 0.8452093091480882, 1.679437397152096, 3.766835247525949, 5.595602401396846, 2.205484490483977, 2.37285418582325
Start iteration 56, value: 0.00045537288941837055, best: 0.00045537288941837055, dval: 0.007122628287318106
4.784840622927583, 0.6726135754170124, 1.0880211911519497, 1.341965989301688, 4.816404129162195, 2.3781713222464727, 4.7887324981091615, 2.814401674269085, 1.9388266467638666, 4.3985453339810565, 3.5478790042988053, 5.41541025160711, 1.4699907197199038, 2.821894261182256, 2.797731456704198, 1.669417596992801, 4.054817211769463, 1.386137994752992, 1.9600857613313485,

Start iteration 74, value: 8.853322369445447e-5, best: 8.853322369445447e-5, dval: 0.0015776198588379753
4.807968605310879, 0.673731771018511, 1.0759868181528593, 1.3489199086837984, 4.816394365312527, 2.3817426665872143, 4.8274733689429725, 2.821257514519867, 1.9388266465142536, 4.38910567410205, 3.5479781759633133, 5.419918610728311, 1.4702537345391065, 2.8205097112209594, 2.7977301926541926, 1.666167801835594, 4.054817211769463, 1.386237166417457, 1.9730158198837593, 5.334429858349845, 1.9736767758019136, 0.8532020182614234, 1.679437397152096, 3.779026323672977, 5.597879209742034, 2.205484490483977, 2.3714395855740533
Start iteration 75, value: 8.840338327739263e-5, best: 8.840338327739263e-5, dval: 0.0034309244078700174
Start iteration 76, value: 0.00010989172769824928, best: 8.840338327739263e-5, dval: 0.006015273385350711
Big value increase: val: 0.00010989172769824928, e.val: 8.840338327739263e-5, max(α): 4.508689091875444
4.808231234871444, 0.6739274893394199, 1.074246606118742

...

4.812743214549929, 0.6743844180854233, 1.0622497445746797, 1.3394384584348533, 4.801614285669488, 2.3811473429652854, 4.828444777072156, 2.830662530341734, 1.9388266465203474, 4.396125412797724, 3.5532928168788596, 5.448971093733568, 1.4885731348665836, 2.8196477188619915, 2.7908279123765225, 1.653684274613082, 4.054817211769463, 1.391551807332726, 1.9828675269940692, 5.3282898846655655, 1.973676774374412, 0.8553016103990017, 1.679437397152096, 3.7794227549232424, 5.596709456332106, 2.205484490483977, 2.372764379528946
Start iteration 130, value: 9.799681039720389e-6, best: 9.799681039720389e-6, dval: 0.0010022892376303216
Start iteration 131, value: 1.0341712004377782e-5, best: 9.799681039720389e-6, dval: 0.0011861697575017427
4.812769609774236, 0.6743843994788771, 1.061937008479879, 1.33940182716365, 4.801083264057427, 2.38114965898172, 4.828444772979098, 2.830697111684538, 1.9388266465203474, 4.396129410935836, 3.553556540900558, 5.448102857030864, 1.4893221529481842, 2.819691467121

4.812803430493045, 0.6743844586768806, 1.061913559690671, 1.3365281989875608, 4.796759564335209, 2.382532780710405, 4.828444759577672, 2.8307109561208947, 1.9388266465203476, 4.397693969641388, 3.554537024214321, 5.450222711032164, 1.4899675952365228, 2.8196899483268667, 2.7895365557219165, 1.6489459829418114, 4.054817211769463, 1.3927960146682659, 1.9836996420229673, 5.328399801553295, 1.9736767743744095, 0.8552770660165956, 1.679437397152096, 3.7794227888981, 5.596709286563323, 2.205484490483977, 2.3738402502548457
Start iteration 149, value: 4.5412668661713276e-6, best: 4.5412668661713276e-6, dval: 0.0004911492300686826
4.812803432716033, 0.6743845073069737, 1.06190480069969, 1.3365160073332996, 4.796711819856309, 2.3825348161466176, 4.828444759577684, 2.830710948030305, 1.9388266465203476, 4.397777819340747, 3.5542346157968128, 5.451964135062621, 1.4900030932328636, 2.8196889981569373, 2.7894083378735925, 1.6487989517573434, 4.054817211769463, 1.3924936062507218, 1.9837717632049168

4.813405757977841, 0.6743845691638071, 1.0610784325462228, 1.3341611686065171, 4.79451973072987, 2.3829332617226737, 4.828444759577808, 2.8307141353743956, 1.9388266465203476, 4.399310698181022, 3.555082484701971, 5.45417920338413, 1.4919119726712144, 2.819528577429173, 2.7877532859340284, 1.6482515456783455, 4.054817211769463, 1.393341475156005, 1.9840502826621296, 5.328245574604161, 1.973676774374408, 0.8554128325581468, 1.679437397152096, 3.779422789345459, 5.596709286874042, 2.205484490483977, 2.373838998509025
Start iteration 165, value: 2.0856218161125833e-6, best: 2.0856218161125833e-6, dval: 0.00019449339047836536
Start iteration 166, value: 7.270330364217929e-6, best: 2.0856218161125833e-6, dval: 0.002152945699989528
Big value increase: val: 7.270330364217929e-6, e.val: 2.0856218161125833e-6, max(α): 98.3841051201878
4.813405260563192, 0.6743845867680974, 1.0608540894822518, 1.3339585367045217, 4.79447439357423, 2.3829331237007723, 4.82844475957782, 2.83071427195578, 1.9388266

Start iteration 184, value: 1.130466715936046e-6, best: 1.130466715936046e-6, dval: 0.00017844288508050976
4.813478347055735, 0.6743846879390838, 1.0604660951956943, 1.3328012947423495, 4.792908365327471, 2.3828496822958156, 4.828445197534801, 2.830851312042558, 1.9388266465203476, 4.399785328343167, 3.555879491459893, 5.456339566668852, 1.4934293617165577, 2.8186314882841237, 2.787533162845138, 1.6471759907465608, 4.054817211769463, 1.3941384819140783, 1.9840501478695203, 5.328348139823134, 1.9736767743744061, 0.8555691536297438, 1.679437397152096, 3.779422789361957, 5.5967092868689186, 2.205484490483977, 2.3738389956444377
Start iteration 185, value: 1.0653769264859258e-6, best: 1.0653769264859258e-6, dval: 0.00015819266192601544
Start iteration 186, value: 3.842310329414677e-6, best: 1.0653769264859258e-6, dval: 0.001566356940583824
Big value increase: val: 3.842310329414677e-6, e.val: 1.0653769264859258e-6, max(α): 217.6721850404648
Start iteration 187, value: 1.0970284175927444e-6

(6.299092356064259e-7, Number[4.813487223740014, 0.674384839557376, 1.0596550966311384, 1.3319723292897854, 4.790284765266623, 2.3828375204097223, 4.828455418293117, 2.831066430458256, 1.9388266465203476, 4.400060036287338  …  1.3942192074432538, 1.984050147868628, 5.328342335771315, 1.9736767743744086, 0.8559359230266543, 1.679437397152096, 3.7794227893550043, 5.596709286865936, 2.205484490483977, 2.3738389956405492], 192)

We define the function to check if we found correct solution.

In [13]:
function check_err_of_ansact(param, check_qc=nothing)
    # Generate ansact from previous step
    ansact = generate_ansact()
    setparameters!(ansact, param)
    inv_ansact = inv(ansact)
    bindparameters!(inv_ansact)

    # Generate qc
    step_qc = generate_empty_circuit()
    # append
    append!(step_qc, inv_ansact)
    if check_qc != nothing
        append!(step_qc, check_qc)
    end
    # Add measures
    addMeasuresOS(step_qc)
    
    return check_circuits_err(qc_full, step_qc)
end

check_err_of_ansact (generic function with 2 methods)

Check if step one finish in proper way.

In [14]:
check_err_of_ansact(step1_start_params, step1_check_qc)

6.211940728223519e-5

Is worth seeing the code of qderivative function available in QuantumCircuits library: https://github.com/Adgnitio/QuantumCircuits.jl/blob/main/src/Execute/Execute.jl

This method calculates the loss function, and the derivative of the loss function using the parameter shift rule (_"Quantum Circuit Learning"_ https://arxiv.org/abs/1803.00745, _"Evaluating analytic gradients on quantum hardware"_ https://arxiv.org/abs/1811.11184) and additionally, it performs the measured error mitigation as described in the notebook [01_solution](01_solution.ipynb). The _Zero Noise Extrapolation_ method is not included because I haven't known about it during the implementation but I'm going to add it in the future.

The parameter for step 1 was found on the **real device**. The code used in training can be found in [sol_step1](src/Challenge/steps/sol_step1.jl) (the code works in exactly the same way as the code above but has had more logging and checks). The log of execution of the code on a real device can be found in [sol_res_step1_Jakarta_1](src/Challenge/steps/sol_res_step1_Jakarta_1.txt) and [sol_res_step1_Jakarta_2](src/Challenge/steps/sol_res_step1_Jakarta_2.txt).

It is worth noting that the printing of the solution can be used to stop the current optimization process and start it again from the previously founded the best solution. This is not necessary for the Julia simulator but in the real device (and even on the simulator) some noises can impact the calculation of derivatives. If we can execute gradient descent many times this doesn't matter because finally, gradient descent finds the proper direction but on the real device, we would like to have as few circuits run as possible.

# 4. Step 2
We do in similar way as in step 1 but we in the addition we add the inverse of the ansact with parameters from step 1.

First we generate the check circuit and the 2 trotter step circuit which will be used in steps from 2 to 5.

In [15]:
# The parameters from previous step
step1_params = step1_start_params
# The parameter wich I found on real deviece
#step1_params = [6.490463894513841, 3.434943430067894, 5.375354561541277, 4.7906997798464115, 2.849325024133124, 2.7673620360366824, 1.4940499436354768, 4.487752591524359, 3.2948988322626382, 2.0225225509500486, -1.2343170007923738, 1.0431091490261049, 1.8837296276360387, 1.2169677999557502, 5.940807695305212, 5.421210358780618, 0.3173688878579456, -6.965567540712155, 4.835967484868803, 3.0127993733543517, 4.4620544822135475, 1.378337396849606, 1.980959799643362, 3.6789144092845376, 5.417860886313657, 1.8687391611628605, 5.092938820960128]

step2_check_qc, _, _, _ = generate_circuit(trotter_steps, 6, t, params, params2, params3, init=false)
trotter_step_qc_exp, _, _, _ = generate_circuit(trotter_steps, st, t, params, params2, params3, init=false)
trotter_step_qc_exp

                                                                         »
q_0: ────────────────────────────────────────────────────────────────────»
       ┌───────────────────────┐  ┌───┐┌──────────┐                 ┌───┐»
q_1: ──┤ U3(π,-4.3798,-1.7351) ├──┤ X ├┤ Rz(8π/5) ├──■──────────────┤ X ├»
       └───────────────────────┘  └─┬─┘└──────────┘  │              └─┬─┘»
q_2: ───────────────────────────────┼────────────────┼────────────────┼──»
        ┌──────────────────────┐    │  ┌──────────┐┌─┴─┐┌──────────┐  │  »
q_3: ───┤ U3(π,0.45277,4.6676) ├────■──┤ Ry(7π/5) ├┤ X ├┤ Ry(8π/5) ├──■──»
        └──────────────────────┘       └──────────┘└───┘└──────────┘     »
q_4: ────────────────────────────────────────────────────────────────────»
     ┌───────────────────────────┐                                       »
q_5: ┤ U3(6.2835,0.084986,0.398) ├───────────────────────────────────────»
     └───────────────────────────┘                                       »
q_6: ────────────────────

Next, we define the helper method which create the optimization circuit for steps from 2 to 5.

In [16]:
function generate_step_ansact(prev_step_params)
    # Generate ansact from previous step
    prev_step_ansact = generate_ansact()
    setparameters!(prev_step_ansact, prev_step_params)
    prev_step_inv_ansact = inv(prev_step_ansact)
    bindparameters!(prev_step_inv_ansact)

    # Generate asnact for current step
    ansact = generate_ansact()
    start_params = getRandParameters(ansact)
    # Set parameters
    setparameters!(ansact, start_params)

    # Generate step qc
    step_qc = generate_empty_circuit()
    # append
    append!(step_qc, prev_step_inv_ansact)
    append!(step_qc, trotter_step_qc_exp)
    append!(step_qc, ansact)
    # Add measures
    addMeasuresOS(step_qc)

    return start_params, step_qc
end

generate_step_ansact (generic function with 1 method)

We generate the optimization circuit.

In [17]:
step2_start_params, step2_qc = generate_step_ansact(step1_params)
step2_qc

                                                                          »
 q_0: ────────────────────────────────────────────────────────────────────»
      ┌─────────────────────────────┐                                     »
 q_1: ┤ U3(-1.6454,-1.3942,-4.0548) ├─────────────────────■───────────────»
      └─────────────────────────────┘                     │               »
 q_2: ────────────────────────────────────────────────────┼───────────────»
      ┌──────────────────────────────┐                  ┌─┴─┐             »
 q_3: ┤ U3(-0.85594,-3.7794,-1.6794) ├──■───────────────┤ X ├─────────────»
      └──────────────────────────────┘  │               └───┘             »
 q_4: ──────────────────────────────────┼─────────────────────────────────»
      ┌─────────────────────────────┐ ┌─┴─┐┌─────────────────────────────┐»
 q_5: ┤ U3(-5.5967,-2.3738,-2.2055) ├─┤ X ├┤ U3(-1.9841,-1.9737,-5.3283) ├»
      └─────────────────────────────┘ └───┘└─────────────────────────────┘»
 q_6: ──────

And optimization function object.

In [18]:
step2_of = create_op(useExecutePlace, step2_qc)
nothing

Now we can find the best parameters.

In [19]:
val, xparams, itr = gradientDescent(step2_of, step2_start_params, α=0.01, maxItr=200,
                              argsArePeriodic=true, isExpectedZero=true, ϵ=1e-6, debug=true, useBigValInc=true)

4.9227046788966895, 5.709577086141721, 5.371418436569134, 4.4680750872355945, 2.4095852537928617, 2.2665333116921045, 4.080004445248484, 1.8557590431394653, 5.964758215330287, 0.05942178447682602, 3.1220847767617115, 5.524146916129699, 5.274863394310612, 4.781551748111868, 5.599211399446277, 3.754636178810669, 4.3026714783343705, 0.09605982671652148, 2.3573497227344826, 5.0316979842629594, 0.47239594972645516, 3.3664648770932057, 4.105812611546052, 2.6768343830712893, 2.432851763497286, 2.1101473732545237, 3.133823101551829
Start iteration 1, value: 2.6031719023362183, best: 2.6031719023362183, dval: 1.9157182744536865
4.935436451540135, 5.720242600966794, 5.364045180596615, 4.47662745506283, 2.422524256375239, 2.267007961299595, 4.079555809453475, 1.8366018603949283, 5.9716748153048185, 0.05169177453105866, 3.132511252356229, 5.534812430954773, 5.27740600966051, 4.7743091797654165, 5.586713020203378, 3.7480524712406376, 4.3026714783343705, 0.10648630231103882, 2.357779478080204, 5.031

Start iteration 17, value: 0.05533414269274163, best: 0.05533414269274163, dval: 0.053575531857659765
5.471765946703846, 5.857857238676798, 4.660787898108555, 4.759782376673825, 2.576726540269303, 2.577018867651478, 4.104440778910619, 1.1366913476818081, 6.291220584410142, -0.7432255404382607, 3.511650593202594, 5.6724270686648115, 5.0215833163195285, 4.638829222312166, 4.6479554121256825, 3.7674178559475555, 4.3026714783343705, 0.48562564315747736, 2.361098358628495, 5.059885677558399, 0.518240027088929, 3.8949660553140815, 4.105812611546052, 7.389781575757946, 2.708490577761166, 2.1101473732545237, 3.8390955338004846
Start iteration 18, value: 0.050838058961127294, best: 0.050838058961127294, dval: 0.049206598711005765
5.4749635640609196, 5.856899759624105, 4.653108139645444, 4.759782895511515, 2.7312497905762276, 2.5771525584170747, 4.103685218463097, 1.1366940956751834, 6.291195732573839, -0.7748139860348299, 3.5131061524836946, 5.671469589612119, 4.9772191778186805, 4.631754560589

5.4749635640609196, 4.108404857031432, 5.006841083354669, 4.759742349129297, 2.586358681793133, 2.3774359573642463, 4.326186745385971, 1.1366944397354333, 6.303063646353919, 0.7660445102415683, 3.526653221185626, 3.9229746870233404, 5.059619284627093, 4.045002812286845, 4.6843641819620645, 2.462580494374117, 4.3026714783343705, 0.5006282711401933, 2.4739555589669324, 5.059885676475649, 0.03818572577452915, 3.9863086586218324, 4.105812611546052, 7.390062585114294, 2.7024505015626836, 2.1101473732545237, 4.37463724930356
Start iteration 49, value: 0.02801267226434315, best: 0.02801267226434315, dval: 0.07839352091893773
5.4749635640609196, 4.095209592997905, 4.989494945760377, 4.7594979212165445, 2.585958935008302, 2.3756671956519004, 4.329918047544828, 1.13669444010578, 6.303067392894435, 0.7782556453163935, 3.526857085981061, 3.9097794229898235, 5.071346578720894, 4.045021960912194, 4.684344649652523, 2.46505782463028, 4.3026714783343705, 0.5008321359356226, 2.4776241561705716, 5.05988

5.4749635640609196, 4.020966576052468, 5.124895572943736, 4.464466680667968, 2.6464338920480395, 2.356891662624661, 4.225595925861121, 1.1368977145917576, 6.30306678697573, 0.8745143782671279, 3.5375907134038465, 3.8053265411937187, 5.055900119816438, 3.9359983450125546, 4.6820249776464005, 2.4189338802777973, 4.3026714783343705, 0.5115657633581028, 2.704043955126257, 5.059885679832141, -0.006604585065323038, 3.9983377961979407, 4.105812611546052, 7.390062584538585, 2.817018953522642, 2.1101473732545237, 4.418824864846873
Start iteration 70, value: 0.01217556250200987, best: 0.01217556250200987, dval: 0.021882278887279533
Start iteration 71, value: 0.06597753158846846, best: 0.01217556250200987, dval: 0.18373393163679339
Big value increase: val: 0.06597753158846846, e.val: 0.01217556250200987, max(α): 773.3813308380725
Start iteration 72, value: 0.013162532872979617, best: 0.01217556250200987, dval: 0.03277903813883354
Start iteration 73, value: 0.023879858989869062, best: 0.0121755625

5.4749635640609196, 4.021229283967319, 5.1060823025904245, 4.352420234304228, 2.642806381798004, 2.310394046531661, 4.2252304087100025, 1.1777416591623362, 6.30332889744965, 0.8769541411684688, 3.4981817248950313, 3.8250467567262008, 5.069810741303543, 3.7867347765100194, 4.682025526093469, 2.4114987480096985, 4.3026714783343705, 0.40839322911398396, 2.6958238735263667, 5.059885679945371, -0.10415273850654745, 3.995000497309855, 4.105812611546052, 7.3900625845419174, 2.8253255111001154, 2.1101473732545237, 4.695309258884238
Start iteration 91, value: 0.009501900993139223, best: 0.009501900993139223, dval: 0.018192697345896285
5.4749635640609196, 4.021346992591734, 5.105555307988091, 4.333377653606686, 2.6428071580152057, 2.277464562716715, 4.22524371121991, 1.177742652825693, 6.303810971369146, 0.8795823966647938, 3.4905966562784476, 3.831967273652104, 5.063893941148164, 3.7916829407380455, 4.6820255311525445, 2.397392071978289, 4.3026714783343705, 0.37870593370023203, 2.69503206931133

...

Start iteration 131, value: 0.00608210394210907, best: 0.00608210394210907, dval: 0.018679680396158
Start iteration 132, value: 0.0067685875074364075, best: 0.00608210394210907, dval: 0.023071515823163413
Big value increase: val: 0.0067685875074364075, e.val: 0.00608210394210907, max(α): 149.62609604750554
Start iteration 133, value: 0.0060899417946648745, best: 0.00608210394210907, dval: 0.013056183605577365
5.4749635640609196, 4.0233586657896465, 5.082516480794396, 4.2986522185872085, 2.6467449805774126, 2.1568201052055276, 4.280722956574109, 1.318724116384273, 6.400989246004196, 0.8879278984921467, 3.4876198932045277, 3.93953875350277, 5.118605665992023, 3.763901591385789, 4.682031550046931, 2.3944681488148083, 4.3026714783343705, 0.36705514322735633, 2.6815422574619463, 5.067365569459513, -0.258997199486588, 4.034473376538166, 4.105812611546052, 7.389600873167691, 2.825287948144547, 2.1101473732545237, 5.114294407451563
Start iteration 134, value: 0.006054630531656467, best: 0.0060

Start iteration 150, value: 0.003512191183880151, best: 0.003512191183880151, dval: 0.013067619570465733
5.4749635640609196, 4.028814773166884, 5.096031234382908, 4.285223097768949, 2.650248774288923, 2.0257736832740365, 4.321139853939861, 1.4184162432768257, 6.455402659177665, 0.9065463597539721, 3.492191956845052, 3.9805704865516733, 5.184088007551595, 3.7639015319868423, 4.67827686763383, 2.372360007455764, 4.3026714783343705, 0.3849498007657556, 2.6786996888689365, 5.076548547285747, -0.2979935014247475, 4.0905685333334825, 4.105812611546052, 7.309390296274963, 2.8252879736545324, 2.1101473732545237, 5.197067575456142
Start iteration 151, value: 0.003268360882735482, best: 0.003268360882735482, dval: 0.017467887846801512
Start iteration 152, value: 0.0038824112270294292, best: 0.003268360882735482, dval: 0.02528777318850537
Big value increase: val: 0.0038824112270294292, e.val: 0.003268360882735482, max(α): 8.93160117619169
5.4749635640609196, 4.028924357377037, 5.096031184291165, 

Start iteration 168, value: 0.001466819984512213, best: 0.001466819984512213, dval: 0.006266052087468901
5.4749635640609196, 4.039280893018613, 5.09603077848515, 4.270140102770224, 2.650289753188709, 1.8957158334497326, 4.381946776870047, 1.4775999908523418, 6.474858896988838, 0.9069333498622151, 3.4944364142383426, 4.033024963484439, 5.232064249985815, 3.763901531987811, 4.671968340906609, 2.359323176772088, 4.3026714783343705, 0.3937344108680229, 2.6646407163044077, 5.080796985539527, -0.2985227092669379, 4.163089955607621, 4.105812611546052, 7.226359577910387, 2.8252879567169504, 2.1101473732545237, 5.3959164945396
Start iteration 169, value: 0.001372383120772194, best: 0.001372383120772194, dval: 0.008112762550126433
Start iteration 170, value: 0.003461012738307319, best: 0.001372383120772194, dval: 0.03192318944542956
Big value increase: val: 0.003461012738307319, e.val: 0.001372383120772194, max(α): 79.27853231673812
5.4749635640609196, 4.038747576629305, 5.096030685373102, 4.270

5.4749635640609196, 4.044566924419757, 5.096057950360827, 4.248625243749637, 2.6502897334342936, 1.8484993489504686, 4.409939822277638, 1.542840677716615, 6.516754061343819, 0.914732341922331, 3.4987091507913033, 4.060171657547652, 5.263597114211061, 3.7639022628342795, 4.6719728949053145, 2.3573735293198244, 4.3026714783343705, 0.4104575283594306, 2.65699587479074, 5.080798211921694, -0.2985221634264903, 4.196302602590761, 4.105812611546052, 7.189341221285587, 2.825287944071892, 2.1101473732545237, 5.498855205142556
Start iteration 187, value: 0.0005416254871607617, best: 0.0005416254871607617, dval: 0.003805409772515485
Start iteration 188, value: 0.0006394401995061195, best: 0.0005416254871607617, dval: 0.010060382866575013
Big value increase: val: 0.0006394401995061195, e.val: 0.0005416254871607617, max(α): 23.146831304822733
5.4749635640609196, 4.044938426742101, 5.096057993118556, 4.246127368215749, 2.650289730606094, 1.8464733507166782, 4.4099545361207415, 1.5494814668338126, 6.

(0.0002470047866310329, Number[5.4749635640609196, 4.045751342391959, 5.096029812194003, 4.221869767826673, 2.650261859358073, 1.8136370592645004, 4.41044904991014, 1.5671941939730467, 6.5489914100770505, 0.9128024175148577  …  0.4047033606385368, 2.6569958656954387, 5.080798207420764, -0.29851786123827206, 4.223751039042287, 4.105812611546052, 7.173416333123011, 2.8185346447958057, 2.1101473732545237, 5.549982773381461], 200)

Check if step one finish in proper way.

In [20]:
check_err_of_ansact(step2_start_params, step2_check_qc)

0.004008319711450683

The parameter for step 2 was found on the noisy simulator. The code used in training can be found in [sol_step2](src/Challenge/steps/sol_step2.jl) (the code works in exactly the same way as the code above but has had more logging and checks). The log of execution of the code on a real device can be found in [sol_res_step2_1](src/Challenge/steps/sol_res_step2_1.txt).


# 5. Step 3

In [21]:
# The parameters from previous step
step2_params = step2_start_params
# The parameter wich I found on real deviece
#step2_params = [6.357739083193767, 4.110466988994183, 4.566133708100635, -4.727581603064405, 6.207391247986331, -1.3477711816915519, -0.7698439582879397, 4.677177128084215, -1.88364164245155, -6.961405873009646, 2.3088656324648755, 1.9854073372986245, -2.4954433452385736, 6.326779366664302, -0.7789648567030898, 2.995616831369772, 6.237335880828537, 0.6943207146536418, 0.37027914593259564, 0.9373275950100715, 2.0325673904575456, 1.3926281522358193, 5.042496816647515, 0.3933698184667445, 3.4025273194720507, 0.9766806621264883, 3.8089690519454855]

step3_check_qc, _, _, _ = generate_circuit(trotter_steps, 4, t, params, params2, params3, init=false)

step3_start_params, step3_qc = generate_step_ansact(step2_params)
step3_of = create_op(useExecutePlace, step3_qc)

val, xparams, itr = gradientDescent(step3_of, step3_start_params, α=0.01, maxItr=200,
                              argsArePeriodic=true, isExpectedZero=true, ϵ=1e-6, debug=true, useBigValInc=true)

4.805225569240957, 2.9695478606187184, 1.179304692733162, 3.317141356081148, 4.878668458206215, 5.9731548131298196, 1.916666289947341, 5.403833043228277, 4.5682312501000375, 0.5010002126570912, 0.7366611990033466, 5.9631390048641135, 3.3653159677883595, 3.4875089887858026, 1.0251076788068774, 3.558660104417252, 1.01036057159997, 4.58315911173476, 3.9110279930995806, 0.9212586431918968, 2.35423423484303, 0.4522258784763621, 5.271994385754124, 2.158866225291617, 0.5367059400451121, 4.267699076269527, 1.4434698249705813
Start iteration 1, value: 1.5592618761270511, best: 1.5592618761270511, dval: 0.8941070078460456
4.807635017873043, 2.967207481476163, 1.1794033643138653, 3.316533475781402, 4.872520221212901, 5.978489782246209, 1.9214966818090249, 5.404387254199429, 4.563003016329672, 0.5085063508331609, 0.7360521213255654, 5.960798625721558, 3.37425703786682, 3.4832263730100093, 1.0304337064799556, 3.5674586845890253, 1.01036057159997, 4.582550034056979, 3.9106428258672183, 0.92242774350

Start iteration 27, value: 0.04786815145330754, best: 0.04786815145330754, dval: 0.07456167165460707
Start iteration 28, value: 0.30835065818276974, best: 0.04786815145330754, dval: 0.5711136631414929
Big value increase: val: 0.30835065818276974, e.val: 0.04786815145330754, max(α): 61.68962923435885
4.8669508995668815, 2.786190739137399, 1.1824850504800848, 4.29264351477919, 4.6430319064114896, 5.791634444421263, 2.428326816580765, 5.762408208597813, 4.293350727910139, 0.8623065234039333, 6.80528500168426, 5.779781885890827, 4.45224778810045, 3.434410836895108, 1.3542747243818996, 4.533312712043574, 1.01036057159997, 4.368597680095822, 3.6102553077743313, 0.9371320080567699, 2.1804041240262966, 1.0463550290649546, 5.271994385754124, 2.3684424653118805, 0.756344154445193, 4.267699076269527, 1.1553261550162488
Start iteration 29, value: 0.04545061150508034, best: 0.04545061150508034, dval: 0.09938476002812319
4.866951216660621, 2.784310623014493, 1.1824850497304837, 4.3012718770629235, 4

Big value increase: val: 0.007975512844752168, e.val: 0.004235626508259594, max(α): 10.99465171138366
4.915083816413982, 2.765661229735154, 1.1828227531708648, 4.377922343142991, 4.693558581230832, 5.471910485996407, 2.187478387227743, 5.808866960601547, 4.240977199770261, 0.9330641979617714, 6.919539518325474, 5.759252376924434, 4.288525163999318, 3.2510683184334446, 1.477311723995184, 4.653603104910356, 1.01036057159997, 4.482852196738963, 3.7079170423815127, 0.9371314728238114, 2.1804340861324327, 1.0272422813886495, 5.271994385754124, 2.4580349687150487, 0.5185342530887559, 4.267699076269527, 1.297028724050546
Start iteration 47, value: 0.0032906680947128782, best: 0.0032906680947128782, dval: 0.018703952127971185
4.915083923690427, 2.765691552910916, 1.1828928428112382, 4.3675039858723945, 4.6855497380373885, 5.478219858558242, 2.19498637550903, 5.809005545426797, 4.215446766534799, 0.9329278647407556, 6.9196074968732875, 5.759282700099545, 4.288504314732226, 3.2513930499505452, 1

Start iteration 64, value: 0.0004692926451149741, best: 0.0004692926451149741, dval: 0.00543181520284311
Start iteration 65, value: 0.1363386167869914, best: 0.0004692926451149741, dval: 0.39975901669749636
Big value increase: val: 0.1363386167869914, e.val: 0.0004692926451149741, max(α): 639.7483350118107
Start iteration 66, value: 0.004417060629355366, best: 0.0004692926451149741, dval: 0.06348287234766192
Big value increase: val: 0.004417060629355366, e.val: 0.0004692926451149741, max(α): 1.891122648464256
Start iteration 67, value: 0.0021367475767318073, best: 0.0004692926451149741, dval: 0.0335612889934328
Big value increase: val: 0.0021367475767318073, e.val: 0.0004692926451149741, max(α): 1.7128179599341575
Start iteration 68, value: 0.0014982348820345146, best: 0.0004692926451149741, dval: 0.022646136236487063
Big value increase: val: 0.0014982348820345146, e.val: 0.0004692926451149741, max(α): 1.7647473366482516
Start iteration 69, value: 0.0009021576384218686, best: 0.0004692

Start iteration 89, value: 4.573312478824053e-5, best: 4.573312478824053e-5, dval: 0.0038380387559344697
4.886321011710335, 2.774365484898962, 1.2313835220106006, 4.349861822576451, 4.645157191801053, 5.392712999898225, 2.1599938992674823, 5.772391121422773, 4.129669067798901, 0.9315131870730431, 6.920226269265224, 5.771115857534831, 4.306625639859947, 3.2848160323953985, 1.5812183880355748, 4.643434886570159, 1.01036057159997, 4.483538947678722, 3.7080841837601146, 0.937137931766291, 2.1804421360906865, 1.00384551883821, 5.271994385754124, 2.5767094312405825, 0.3900328578017677, 4.267699076269527, 1.2505801176629003
Start iteration 90, value: 4.473888088321054e-5, best: 4.473888088321054e-5, dval: 0.002993718058541456
4.883220645770284, 2.774619294612644, 1.231395166377799, 4.3502885435956635, 4.6451816593547814, 5.387411015757073, 2.1599933116730115, 5.7723810027974665, 4.1296696764985485, 0.9314733834535778, 6.920226269363982, 5.771351136020458, 4.306544346741988, 3.2849583338543455

Start iteration 111, value: 3.352917051230731e-6, best: 3.352917051230731e-6, dval: 0.0005910878249378859
4.879855860884877, 2.7758742320730287, 1.2410447533575861, 4.349518748188301, 4.643024248695991, 5.384391075770513, 2.15999279167398, 5.772584133744256, 4.129666972125576, 0.9378614548136593, 6.9202262697953, 5.772903504017905, 4.306922225757426, 3.283874293414242, 1.5827652523991627, 4.637468881330978, 1.01036057159997, 4.483538948208798, 3.708129765990526, 0.9371375388683612, 2.1804421581212736, 1.002742891847334, 5.271994385754124, 2.585257866759617, 0.3742720561598693, 4.267699076269527, 1.2574536555369713
Start iteration 112, value: 2.5388621444856047e-6, best: 2.5388621444856047e-6, dval: 0.0007733427714936569
4.879662232929161, 2.7758742506579286, 1.2400260477573748, 4.349017222047026, 4.6430157711762625, 5.384389852437033, 2.1599928412906277, 5.772584130864105, 4.129667158567638, 0.9379404303743827, 6.920226269795296, 5.772903550479499, 4.30694398140235, 3.2841052365829477,

(6.364094419322293e-7, Number[4.879086586345206, 2.775874184820651, 1.2404853028028002, 4.349280582246125, 4.642165527202472, 5.384388192560105, 2.159993050653345, 5.7725841302580605, 4.129667158314952, 0.9380795701131336  …  4.483538948208801, 3.7081315763430136, 0.9371375208513264, 2.180442163223591, 1.0027400067414232, 5.271994385754124, 2.585222779300661, 0.3724813288127018, 4.267699076269527, 1.2551822194654445], 115)

In [22]:
check_err_of_ansact(step3_start_params, step3_check_qc)

0.004345722576117811

The parameter for step 3 was found on the noisy simulator. The code used in training can be found in [sol_step3](src/Challenge/steps/sol_step3.jl) (the code works in exactly the same way as the code above but has had more logging and checks). The log of execution of the code on a real device can be found in [sol_res_step3_1](src/Challenge/steps/sol_res_step3_1.txt).


# 6. Step 4

In [23]:
# The parameters from previous step
step3_params = step3_start_params
# The parameter wich I found on real deviece
#step3_params = [5.189896155542379, 4.707110397329062, 4.325905810068413, -0.8159924212917966, 3.110011220351916, 4.667709359716189, 1.0215152198560669, 5.116208100394577, 4.395063842373562, 4.370201901371649, 4.9135180721869425, 4.949975295863072, 4.637313944153916, 4.042830477313872, -0.6901894825683017, 1.1827639351923742, 3.0974447481049743, 4.022975117724089, 0.705887140029585, 3.514630564645274, 1.755049621148228, 4.151136862582433, 5.722357530716369, 0.8667811089902828, 0.3630963695624429, 0.1785816271535451, 3.4571312587872733]

step4_check_qc, _, _, _ = generate_circuit(trotter_steps, 2, t, params, params2, params3, init=false)

step4_start_params, step4_qc = generate_step_ansact(step3_params)
step4_of = create_op(useExecutePlace, step4_qc)

val, xparams, itr = gradientDescent(step4_of, step4_start_params, α=0.01, maxItr=200,
                              argsArePeriodic=true, isExpectedZero=true, ϵ=1e-6, debug=true, useBigValInc=true)

1.1902958633134078, 6.103438100129545, 1.6629204988394812, 0.20959675383793908, 4.497528064913114, 2.3936879549524956, 2.4098683969197494, 5.936336615764863, 2.243981665182, 3.6672318872138305, 1.3094115057042055, 1.1690522062680186, 6.215257027862909, 2.4746626298075567, 4.391509336457032, 0.7686496656807797, 4.312918048353797, 1.95092178721054, 4.723056947869843, 5.371258316605843, 1.0321290459078003, 5.664792077872366, 1.1836816967163313, 0.8750898833701132, 1.0365995333925504, 1.0732163200297484, 1.0538818017793246
Start iteration 1, value: 1.4269841928927969, best: 1.4269841928927969, dval: 0.41100037884239204
1.1867315149204813, 6.10250591857396, 1.6630243921228278, 0.20922934852873157, 4.497464564079767, 2.393973767649296, 2.4106714732538608, 5.939439724515138, 2.2437097571106537, 3.664011282893171, 1.310235409028616, 1.1681200247124333, 6.212674847413473, 2.4711743085505398, 4.387886103314412, 0.7719469229478071, 4.312918048353797, 1.9517456905349504, 4.7251339071382255, 5.3738

1.013240497990849, 6.102931375552077, 1.977533872319552, -0.14457073906146173, 4.345144512189104, 2.1714422960388484, 2.4308916925605795, 6.701400910403363, 2.12359395161084, 3.959911289355314, 1.3136482934218827, 1.1685454816904937, 4.667256066068283, 2.6653620881101014, 3.838301931889197, 0.8702760228460908, 4.312918048353797, 1.955158574928217, 4.789342860506067, 5.066344168249727, 1.11122863960497, 4.96239991285026, 1.1836816967163313, 0.38857115837851763, 1.1652243020911401, 1.0732163200297484, 0.8611248151829333
Start iteration 20, value: 0.12782029040501977, best: 0.12782029040501977, dval: 0.23819823443600904
1.0141661966192592, 6.103000020649788, 1.9873434684215545, -0.15203667389975956, 4.337868908191316, 2.165304714821735, 2.39153376299805, 6.701420781045713, 2.121279067663356, 3.9599574435844924, 1.3136307486257386, 1.168614126788206, 4.665784297011074, 2.6396995724410552, 3.838842297870918, 0.8718320195957525, 4.312918048353797, 1.9551410301320729, 4.79439152396431, 4.9494

1.014159294177741, 6.103145121771643, 2.0693331262391585, -0.44669742023481296, 4.3181446675886415, 2.170443241117638, 1.726979242198526, 6.658533306522303, 2.0017818133365366, 3.935993871810765, 1.2986437171915652, 1.1687632730929198, 4.566747933266451, 2.76902254621927, 3.8126688444463706, 0.9989471067698787, 4.312918048353797, 1.6186201735397847, 4.573865586313516, 4.934729335208944, 1.132602944629235, 4.847404660336093, 1.1836816967163313, 0.3885712306929072, 1.1652233362862159, 1.0732163200297484, 1.0350821452298395
Start iteration 39, value: 0.0007778933860829431, best: 0.0007778933860829431, dval: 0.004891444997994963
1.0141593258955381, 6.103145941684248, 2.0653807626865386, -0.44669743974765924, 4.314170520467074, 2.170664987376532, 1.7233204113674214, 6.655625087216905, 2.001237351750865, 3.936013874500454, 1.2988638606871568, 1.1687639200396531, 4.566986002518939, 2.769289348357663, 3.812679590479939, 0.9976198821378414, 4.312918048353797, 1.630365391506093, 4.57278656855643

Start iteration 57, value: 0.0003468037566528809, best: 0.0003468037566528809, dval: 0.004588694705769447
1.0141593589734748, 6.103736981308436, 2.0654929213726616, -0.4471611207808027, 4.273278849010887, 2.1719815500849777, 1.6865164493655513, 6.647368608186252, 1.9989076894270759, 3.936146211415782, 1.2993723874385676, 1.1801376859264698, 4.5966162230016625, 2.771983083479777, 3.812969092574542, 0.9917495918214579, 4.312918048353797, 1.657496593002224, 4.542841923615653, 4.959745349151729, 1.1309516163181934, 4.8182520497789705, 1.1836816967163313, 0.38857123187561105, 1.1652233321875691, 1.0732163200297484, 1.0301717764987892
Start iteration 58, value: 0.0003321482660463028, best: 0.0003321482660463028, dval: 0.003639561399230745
Start iteration 59, value: 0.0006080337519892861, best: 0.0003321482660463028, dval: 0.017463613155446506
Big value increase: val: 0.0006080337519892861, e.val: 0.0003321482660463028, max(α): 17.170865443223228
Start iteration 60, value: 0.00038388164858964

Start iteration 75, value: 0.00010780056894634361, best: 0.00010780056894634361, dval: 0.0022621613212556665
1.0141593591229097, 6.106243230980131, 2.065149096382009, -0.4605290434192636, 4.264824472086277, 2.190641292878605, 1.6705795861007926, 6.645744798119125, 1.9989564307373247, 3.9310325374164843, 1.3000798313152764, 1.2227651943951239, 4.598815385991495, 2.7729965331503657, 3.8207462476941325, 0.9891142671873904, 4.312918048353797, 1.6952405313749335, 4.556463204230781, 4.971584041624167, 1.120336221259951, 4.819212952741255, 1.1836816967163313, 0.3926141695381841, 1.1652233322148902, 1.0732163200297484, 1.0244734341235013
Start iteration 76, value: 9.784658421584741e-5, best: 9.784658421584741e-5, dval: 0.0020118836213660393
1.0141593591234488, 6.106699044562034, 2.065036583308627, -0.4612388565311545, 4.26379567128849, 2.190817713697263, 1.6708220929805557, 6.645667670982258, 1.9989564193923597, 3.931003268872867, 1.3002610614620482, 1.228484332369718, 4.599623778089403, 2.775

Start iteration 95, value: 3.453261579890214e-5, best: 2.8005359359231876e-5, dval: 0.002426555157449106
Big value increase: val: 3.453261579890214e-5, e.val: 2.8005359359231876e-5, max(α): 31.079564177296998
Start iteration 96, value: 0.0012454735203175334, best: 2.8005359359231876e-5, dval: 0.03479625303619814
Big value increase: val: 0.0012454735203175334, e.val: 2.8005359359231876e-5, max(α): 205.34782202636978
Start iteration 97, value: 2.9798806619790074e-5, best: 2.8005359359231876e-5, dval: 0.0032790374334744077
Start iteration 98, value: 0.00032493589463307584, best: 2.8005359359231876e-5, dval: 0.017244352919167538
Big value increase: val: 0.00032493589463307584, e.val: 2.9798806619790074e-5, max(α): 0.7407919681607962
1.0141593592800762, 6.10780204330658, 2.0649205730395264, -0.4714713134128181, 4.262234647927939, 2.192398021043209, 1.6609709364547283, 6.64155859518758, 1.998956016505868, 3.9281448295645847, 1.3004507921595092, 1.2465791253893541, 4.600655564834175, 2.775363

Start iteration 113, value: 8.344194051522892e-6, best: 8.344194051522892e-6, dval: 0.0005862007041329623
Start iteration 114, value: 7.020245672215402e-5, best: 8.344194051522892e-6, dval: 0.00790425443041029
Big value increase: val: 7.020245672215402e-5, e.val: 8.344194051522892e-6, max(α): 78.60903456032905
Start iteration 115, value: 0.0007260535240692738, best: 8.344194051522892e-6, dval: 0.02530405471743006
Big value increase: val: 0.0007260535240692738, e.val: 8.344194051522892e-6, max(α): 92.90108080781282
Start iteration 116, value: 1.1193508442359827e-5, best: 8.344194051522892e-6, dval: 0.0018647920344800617
Big value increase: val: 1.1193508442359827e-5, e.val: 8.344194051522892e-6, max(α): 2.128127098135929
1.0141593601078092, 6.1078536444842255, 2.0649205731272215, -0.471681585673762, 4.257948381146325, 2.192386957630158, 1.6551635003363188, 6.637728671336785, 1.99895601614607, 3.9269682055005966, 1.300596949619057, 1.2503154270731305, 4.6055246873170645, 2.77814319138984

Start iteration 134, value: 4.130481440986196e-6, best: 4.130481440986196e-6, dval: 0.0009786079931175425
1.0141593601712156, 6.1078539457934555, 2.0649205731271487, -0.4728181642379842, 4.2548153388258, 2.192470445793108, 1.6523350273573303, 6.636205471634394, 1.9989560158861415, 3.926818769878222, 1.3006864716123903, 1.2503248513701035, 4.6082522219015685, 2.778750556804916, 3.82913292206474, 0.985979718646422, 4.312918048353797, 1.7276063409685898, 4.55113019639635, 4.97369532266442, 1.0949880032007573, 4.812324405119548, 1.1836816967163313, 0.4091448714064273, 1.1652233322141416, 1.0732163200297484, 1.0167569005894184
Start iteration 135, value: 2.936814698310191e-6, best: 2.936814698310191e-6, dval: 0.00043584827615734956
1.0141593601712087, 6.10785401870987, 2.0649205731271505, -0.47281941065564337, 4.254753588555295, 2.192470157937022, 1.6521885030871064, 6.63619356827799, 1.9989560158845514, 3.92681873950059, 1.3006864462932377, 1.2503271367693727, 4.608499910127157, 2.77874904

Start iteration 153, value: 1.269536342783324e-6, best: 1.269536342783324e-6, dval: 0.0002089333746140357
1.0141593601721839, 6.107852583565742, 2.0649205731271385, -0.4746428215629193, 4.252318692598024, 2.1927203234857777, 1.6500214954060788, 6.635361633650155, 1.9989560143448188, 3.926567350621955, 1.3007082836698476, 1.2507648271808702, 4.60973317991395, 2.779028158207458, 3.829132921770756, 0.9857116603381976, 4.312918048353797, 1.7287700643561825, 4.549597820837944, 4.974752016371574, 1.093855308016304, 4.811798234037704, 1.1836816967163313, 0.40994908345096664, 1.1652233322129413, 1.0732163200297484, 1.0157231291675977
Start iteration 154, value: 1.2168459230914368e-6, best: 1.2168459230914368e-6, dval: 0.00017387511671624811
1.0141593601729681, 6.10785260382257, 2.0649205731271385, -0.47464713281068277, 4.252303457663776, 2.192727398231186, 1.6500053835571, 6.635235519634398, 1.9989560027509767, 3.926566978192177, 1.3007098129542287, 1.2507818545240599, 4.609801809045849, 2.779

(9.60445967288338e-7, Number[1.0141593601741943, 6.1078526627937215, 2.064920573127139, -0.47465642378963674, 4.251240591715009, 2.1925456116553206, 1.6494543768230923, 6.634464234741885, 1.9989559977340243, 3.9265652685468955  …  1.730399844951208, 4.548996452158186, 4.974915768140478, 1.093599788219283, 4.811799315313045, 1.1836816967163313, 0.4099473688323421, 1.1652233322106642, 1.0732163200297484, 1.0158575099074163], 158)

In [24]:
check_err_of_ansact(step4_start_params, step4_check_qc)

0.004418574031107449

The parameter for step 4 was found on the noisy simulator. The code used in training can be found in [sol_step4](src/Challenge/steps/sol_step4.jl) (the code works in exactly the same way as the code above but has had more logging and checks). The log of execution of the code on a real device can be found in [sol_res_step4_1](src/Challenge/steps/sol_res_step4_1.txt).


# 7. Step 5

The parameter for step 5 was found on the noisy simulator. The code used in training can be found in [sol_step5](src/Challenge/steps/sol_step5.jl) (the code works in exactly the same way as the code above but has had more logging and checks). The log of execution of the code on a real device can be found in [sol_res_step5_1](src/Challenge/steps/sol_res_step5_1.txt).

In [25]:
# The parameters from previous step
step4_params = step4_start_params
# The parameter wich I found on real deviece
#step4_params = [1.1187171575015713, 0.5330092890891549, 2.586441510958004, 1.5178154678919513, 3.137963883844627, 4.228048049765462, 4.839509162353759, 2.9578752785288516, 4.768574523872119, 1.6545920069530968, 2.9732151650981935, 5.438796728232314, 1.3567895934397545, 2.2177922918734536, 2.0165705795863307, -0.14637716576453905, 0.4175300959136594, 4.311416597020989, 5.65363080884547, 5.1078093853571485, 2.6110312065705705, 3.9648509676298693, 2.989797720639175, 2.5851197335780856, 1.9933603087957705, 0.5174013121704684, 2.1454323563784383]

step5_start_params, step5_qc = generate_step_ansact(step4_params)
step5_of = create_op(useExecutePlace, step5_qc)

val, xparams, itr = gradientDescent(step5_of, step5_start_params, α=0.01, maxItr=200,
                              argsArePeriodic=true, isExpectedZero=true, ϵ=1e-6, debug=true, useBigValInc=true)

4.198660630722049, 3.2025502702597057, 1.1917502905334754, 0.9033608037870118, 5.9266811001056, 2.8333305919269587, 0.31488175486642195, 1.4845040128725302, 3.301634571133797, 4.249316368526539, 0.9080213048543806, 1.061888660170322, 3.979020984781252, 2.144013706418033, 5.310401846736067, 1.7237465199724622, 2.7453419101086176, 0.5583676007774854, 1.1091625423425624, 0.3789424817018281, 3.6222570612006395, 2.647353971009628, 3.9657965800421877, 2.047659979753023, 5.62259659235652, 3.0641695752240024, 0.6047129052124455
Start iteration 1, value: 0.7999674739165794, best: 0.7999674739165794, dval: 0.5511212891644865
4.200311327524355, 3.204175579156683, 1.1932137813207548, 0.8998246369348238, 5.923475118276995, 2.833412001842059, 0.31825741114787715, 1.4814297013607816, 3.300158842884327, 4.252642780653905, 0.9118072849645964, 1.0635139690672988, 3.9749689366623073, 2.142658774499753, 5.311128418024206, 1.723965600420572, 2.7453419101086176, 0.5621535808877012, 1.1146737552342072, 0.380

Big value increase: val: 0.022020195309358403, e.val: 0.018577791812759984, max(α): 107.2079363802558
4.737048492765691, 3.587988531978717, 0.9172039735438631, 0.3977100557478616, 5.444458407688289, 3.951837387742387, 0.7701306811604081, 1.2223245814914594, 3.2404030938074886, 4.4420469052996046, 1.179178737746345, 1.4473269218892586, 3.9191303914189453, 2.161073518747038, 5.329106017303096, 1.6811852610123115, 2.7453419101086176, 0.8295250336694536, 1.4736466192170616, 0.44659078878561187, 4.1603632712601515, 2.464066845789227, 3.9657965800421877, 1.6117951667319743, 5.5197883626425455, 3.0641695752240024, 0.22212624339727274
Start iteration 20, value: 0.017592236653721498, best: 0.017592236653721498, dval: 0.03189316412413165
Start iteration 21, value: 0.018884097743187764, best: 0.017592236653721498, dval: 0.06613951184812969
Start iteration 22, value: 0.025887476149107858, best: 0.017592236653721498, dval: 0.11109314813261079
Big value increase: val: 0.025887476149107858, e.val: 0.

Start iteration 38, value: 0.003948964976455477, best: 0.003948964976455477, dval: 0.025905575485903887
4.719870023707518, 3.588160208534987, 0.9158179426919602, 0.6642279804875059, 5.442616045810705, 3.5936277779325714, 0.8809106211763327, 1.3013825054749535, 3.2404254277837676, 4.516256356461266, 1.1914586672043856, 1.4473624817016968, 3.9487823267228754, 2.2117391043982, 5.417291414651749, 1.7044092205793402, 2.7453419101086176, 0.842529528550936, 1.5090820499066275, 0.4504725097281808, 4.12084305177925, 2.2936421986714954, 3.9657965800421877, 1.6748628995107753, 5.703689189104295, 3.0641695752240024, 0.22212591709783233
Start iteration 39, value: 0.0036352712979464638, best: 0.0036352712979464638, dval: 0.024586700795855232
4.719869280472014, 3.5881144866302814, 0.9196373693608558, 0.6189219895407809, 5.443245901878812, 3.5949984766744194, 0.881521450938624, 1.303401417348458, 3.2404168580550228, 4.537792104812581, 1.1914589077801556, 1.4473524444177635, 3.9564439086034393, 2.21372

4.719605538300655, 3.588193480463816, 1.0006967829394784, 0.6503106011160701, 5.464536173115769, 3.7686595154625806, 0.8821637793307083, 1.3650262699318325, 3.2403544249885874, -1.740365443929135, 1.2045237407106333, 1.4473697858550707, 3.9766646504588743, 2.2536424858078794, 5.491805572991809, 1.7276083348571716, 2.7453419101086176, 0.8570242995012985, 1.5265785707062924, 0.4481359664181766, 4.142896086648785, 2.225110412645737, 3.9657965800421877, 1.6841159555681817, 5.7419165892809945, 3.0641695752240024, 0.2221260225381632
Start iteration 66, value: 0.0006173938883171082, best: 0.0006173938883171082, dval: 0.01211500203284498
4.719605538300537, 3.588194072154952, 1.00070419481569, 0.6631571286338234, 5.464590300909009, 3.7686596014693015, 0.8821634332008773, 1.365020593119828, 3.2403544105686897, -1.7406141758765175, 1.2043246506160354, 1.447369915748895, 3.9766646520020354, 2.2536425958107458, 5.4918050661106115, 1.727238005661599, 2.7453419101086176, 0.8568034231841236, 1.5265535

Start iteration 83, value: 6.113233895674693e-5, best: 6.113233895674693e-5, dval: 0.0023443094546354293
Start iteration 84, value: 0.00825468974825538, best: 6.113233895674693e-5, dval: 0.07517432873332684
Big value increase: val: 0.00825468974825538, e.val: 6.113233895674693e-5, max(α): 1269.429506939017
4.711774740444479, 3.588510167219905, 1.0152797532237787, 0.6886249948084527, 5.487388808732175, 3.8057551094041426, 0.887823530836877, 1.3638631063763682, 3.240342943230312, -1.7346446049017787, 1.2052112380953957, 1.4474393080286994, 3.992431605504405, 2.2837724639628902, 5.503277839179125, 1.733823004288338, 2.7453419101086176, 0.8577870290118776, 1.5267862049672571, 0.44813596641907166, 4.150077362840512, 2.190904464695743, 3.9657965800421877, 1.682564598164702, 5.77018932011256, 3.0641695752240024, 0.22212633854206154
Start iteration 85, value: 5.760477932348797e-5, best: 5.760477932348797e-5, dval: 0.004768627596404423
4.71177051498771, 3.588509230681107, 1.023228200823255, 0.6

Start iteration 105, value: 3.6986225196463112e-6, best: 3.6986225196463112e-6, dval: 0.0006994879714601176
4.711822539347597, 3.5894504105008442, 1.0180923862117928, 0.7000791115473428, 5.484268255348559, 3.8057584694558297, 0.8916422647377686, 1.3653953570339188, 3.2403429284013017, -1.7319466948698927, 1.207170239783962, 1.4474580514821194, 3.989661456455633, 2.287511226416599, 5.507993386576025, 1.732218848008211, 2.7453419101086176, 0.8599604022236526, 1.5295033530259894, 0.4481359659864896, 4.151890532641409, 2.187012518913906, 3.9657965800421877, 1.6853516162769246, 5.781545634059878, 3.0641695752240024, 0.22212811548789443
Start iteration 106, value: 3.2758789378696025e-6, best: 3.2758789378696025e-6, dval: 0.0006430559888781382
Start iteration 107, value: 7.833818835121753e-6, best: 3.2758789378696025e-6, dval: 0.0022989066843073173
Big value increase: val: 7.833818835121753e-6, e.val: 3.2758789378696025e-6, max(α): 11.572819015046086
4.711821177218511, 3.5894504104689418, 1.0

(8.325110472025892e-7, Number[4.711952965244252, 3.589450438231447, 1.0251345436411903, 0.7001235212589622, 5.481406106047752, 3.814412070692628, 0.8929910933806784, 1.3687624490136485, 3.240342804817119, -1.7319278378975753  …  0.8602379979340482, 1.5298076732507906, 0.4481296051274036, 4.151972359378279, 2.186054835828029, 3.9657965800421877, 1.6877303305176357, 5.781313335445093, 3.0641695752240024, 0.2215248575264341], 114)

In [26]:
check_err_of_ansact(step5_start_params)

0.0047718538428913534